In [1]:
import os
import json
import pandas as pd
import re
import random
import matplotlib.pyplot as plt
import numpy as np


In [2]:
#LOGO
train_logo_data = "logo_data/python/train_200_dataset.jsonl"
test_logo_data = "logo_data/python/test_dataset.jsonl"

# Load train and test dataset
with open(f"external/dependencies/{train_logo_data}", 'r') as f:
    train_data = [json.loads(line) for line in f]

with open(f"external/dependencies/{test_logo_data}", 'r') as f:
    test_data = [json.loads(line) for line in f]

In [3]:
# Extract descriptions and programs from train_data and transform into pandas DataFrame
def extract_descriptions_and_programs(data):
    extracted_data = []
    for item in data:
        description = None
        program = None
        for message in item.get('messages', []):
            if message['from'] == 'human':
                description = message['value']
            elif message['from'] == 'gpt':
                program = message['value']
        if description and program:
            extracted_data.append([description, program])
    extracted_data = pd.DataFrame(extracted_data, columns=['Description', 'Program'])
    return extracted_data

# Extract descriptions and programs from train_data
df_train = extract_descriptions_and_programs(train_data)
df_test = extract_descriptions_and_programs(test_data)

#display(df_train)
#display(df_test)

# Append the test data to the train data
df_all = pd.concat([df_train, df_test], ignore_index=True)
#display(df_all)


#####################
# Drop all duplicate rows
df_all = df_all.drop_duplicates(subset=['Description', 'Program'])
#####################
display(df_all)


,Description,Program
0,4 concentric square s,"for i in range(5):\n embed(""""""for j in rang..."
1,6 sided snowflake with a medium line and a med...,"for j in range(6):\n embed(""""""forward(8)\nl..."
2,5 sided snowflake with a medium line and a sma...,"for j in range(5):\n embed(""""""forward(8)\nl..."
3,6 short line s in a row,"for j in range(6):\n embed(""""""forward(2)\nl..."
4,a small triangle connected by a big line to a ...,for i in range(3):\n forward(2)\n left(1...
...,...,...
306,8 sided snowflake with a medium circle and a s...,"for j in range(8):\n embed(""""""penup()\nforw..."
307,5 sided snowflake with 2 small circle s as arms,"for j in range(5):\n embed(""""""penup()\nforw..."
308,3 sided snowflake with a small square and a sm...,"for j in range(3):\n embed(""""""penup()\nforw..."
309,5 sided snowflake with a small 5 gon and a sma...,"for j in range(5):\n embed(""""""penup()\nforw..."


#### Synthetic creation of new Programs

The generated code is in the same pseudo-code format as in the original train and test data. So, it is of type string and can't be execued printing a graphic.

In [ ]:
## Top-level grammar rules for LOGO
# <Program> ::= <Position> | <Shape-Sequence> | <SpecialShape> | <Snowflake> 
# <Shape-Sequence> ::= <Shape> | <Shape> " & " <Shape-Sequence> | <Space> " & " <Shape-Sequence>
# <Sub-Program> ::= <Connected-Sub-Program> | <Separated-Sub-Program>
# <Connected-Sub-Program> ::= "embed("  <Shape-Sequence> ", locals())"
# <Separated-Sub-Program> ::= "embed(" <Space> " & "  <Shape-Sequence> ", locals())"

### POSITION
# <Position> ::= <InARow> | <Concentric>
# <InARow> ::= "for j in range(" <N-Times> "): " <Sub-Program>
# <Concentric> ::= "for j in range(" <N-Times> "): " <Shape> 
#             NOTE: "forward(" <Length> | <Size>" * " <N-Times> ")" # parameter within sub-program

### SHAPE
# <Shape> ::= <Line> | <Polygon> | <Semicircle> | <Circle> 
# <Line> ::= "forward(" <Length> ") left(" <Angle> ")" | "forward(" <Length> ")
# <Polygon> ::= "for i in range(" <Sides> "): forward(" <Length> ") left(" <Angle> = (360/ <Sides>)")" 
# <Semicircle> ::= "for i in range(HALF_INF): forward(" <Size> ") left(EPS_ANGLE)"
# <Circle> ::= <Semicircle> <Semicircle>

### SPECIAL SHAPE
# <SpecialShape> ::= <GreekSpiral> | <Staircase> | <Zigzag> | <Star>
# <GreekSpiral> ::= "for i in range(" <Sides> "): forward(1 * " <Sides> ") left(90.0)"
# <Staircase> ::= "for i in range(" <N-Times> "): forward(" <Length> ") left(90.0) forward(" <Length> ") left(90.0) forward(0) left(180.0)"
# <Zigzag> ::= "for i in range(" <N-Times> "): forward(" <Length> ") left(90.0) forward(" <Length> ") left(270.0)"
# <Star> ::= "for i in range(" <Sides> "): forward(16) left(" <Angle> = (360/ <Sides>)")"

### SPACE
# <Space> ::= "penup() " <Line> " pendown()"

### SNOWFLAKE
# <Snowflake> ::= "for j in range(" <Sides> "): " <Sub-Program> 
#                 "forward(0) left(360 / " <Sides> ")"

### Parameters
# <Sides> ::= "3" | "5" | "6" | "7" | "8"
# <Length> ::= "2" | "4" | "20"          # short/small, medium, big
# <Size> ::= "EPS_DIST" | "EPS_DIST * 2" # small, medium applying to (semi)circle

# <Angle> ::= <Digit> "." <Digit>
# <Sides> ::= "3" | "4" | "5" | "6" | "7" | "8" | "9"
# <N-Times> ::= <Digit> 
# <Digit> ::= "1" | "2" | "3" | "4" | "5" | "6" | "7" | "8" | "9"

In [12]:
import re
class generateLOGOPseudoCode():
    def __init__(self):
        pass

#Basic Shapes
    def generate_line(self, length: int, angle: float = 0.0, left: bool = True) -> str:
        if not (0.0 <= angle <= 360.0):
            raise ValueError("Angle must be between 0.0 and 360.0")
        if angle == 0:
            return f"forward({length})"
        else:
            direction = "left" if left else "right"
            return f"forward({length})\n{direction}({angle})"
            
    def generate_polygon(self, sides: int, length: int, left: bool = True) -> str:
        angle = 360 / sides
        direction = "left" if left else "right"
        return f"for i in range({sides}):\n    forward({length})\n    {direction}({angle})"

    def generate_semicircle(self, size: int, semicircle: bool = True, left: bool = True) -> str:
        EPS_ANGLE = 1.0
        EPS_DIST = 0.03490481287456702  # Using the defined EPS_DIST value
        HALF_INF = 180
        size = EPS_DIST * size

        direction = "left" if left else "right"
        semicircle_code = f"for i in range({HALF_INF}):\n    forward({size})\n    {direction}({EPS_ANGLE})"
        
        if semicircle:
            return semicircle_code
        else:  # circle
            return f"{semicircle_code}\n{semicircle_code}"
            
        
#Special Shapes            
    def generate_greek_spiral(self, size: int, left: bool = True) -> str:
        if not (5 <= size <= 9):
            raise ValueError("A greek-spiral must have at least 5 turns")
        direction = "left" if left else "right"
        return f"for i in range({size}):\n    forward(1 * {size})\n    {direction}(90.0)"

    def generate_staircase(self, n_times: int, length: int, left: bool = True) -> str:
        direction = "left" if left else "right"
        return f"for i in range({n_times}):\n    forward({length})\n    {direction}(90.0)\n    forward({length})\n    {direction}(90.0)\n    forward(0)\n    {direction}(180.0)"

        
    def generate_zigzag(self, n_times: int, length: int, left: bool = True) -> str:
        direction = "left" if left else "right"
        return f"for i in range({n_times}):\n    forward({length})\n    {direction}(90.0)\n    forward({length})\n    {direction}(270.0)"
    
    # this is how star seems to be generated in the ReGAL paper but it doesn't result in symetric stars (it also fails for stars with 6 points completely) Maybe thats the reason why they only inclued stars with 5 points
    #def generate_star(self, sides: int, left: bool = True) -> str:
    #    if not (5 <= sides <= 9):
    #        raise ValueError("Stars must have between 5 and 9 points")
    #    angle = 360 / sides
    #    direction = "left" if left else "right"
    #    return f"for i in range({sides}):\n    forward(16)\n    {direction}({angle})"
    def generate_star(self, sides: int, left: bool = True) -> str:
        if not (5 <= sides <= 9):
            raise ValueError("Stars must have between 5 and 9 points")
        direction = "left" if left else "right"
        if sides == 6:
            return f"for i in range(3):\n    forward(10)\n    {direction}(120.0)\npenup()\n{direction}(90.0)\nforward(6)\n{direction}(210)\npendown()\nfor i in range(3):\n    forward(10)\n    {direction}(120.0)"
        elif sides == 8:
            angle = ((360.0*3) / sides)
            return f"for i in range(8):\n    forward(10)\n    {direction}({angle})"
        else: #odd-numbers
            angle = (180.0 - (180.0 / sides))
            return f"for i in range({sides}):\n    forward(10)\n    {direction}({angle})"

    def generate_space(self, length: int, angle: float = 0.0, left: bool = True) -> str:
        if not (0.0 <= angle <= 360.0):
            raise ValueError("Angle must be between 0.0 and 360.0")
        if angle == 0:
            return f"penup()\nforward({length})\npendown()"
        else:
            direction = "left" if left else "right"
            return f"penup()\nforward({length})\n{direction}({angle})\npendown()"
            
#Position
    def in_a_row(self, n_times: int, sub_program: str, left: bool = True) -> str:
        #indented_program = re.sub(r"(\n)", r"\1    ", sub_program)
        direction = "left" if left else "right"
        return f"for j in range({n_times}):\n    {sub_program}\n    penup()\n    forward(2)\n    {direction}(0.0)\n\n    pendown()"

    def concentric_semicircle(self, n_times: int, semicircle: bool = True, left: bool = True) -> str:
        EPS_ANGLE = 1.0
        EPS_DIST = 0.03490481287456702
        HALF_INF = 180
        direction = "left" if left else "right"
        semicircle_code = (
            f"for i in range({HALF_INF}):\n        forward({EPS_DIST} * j)\n        {direction}({EPS_ANGLE})"
        )
        if semicircle:
            return f"for j in range({n_times}):\n    {semicircle_code}"
        else:
            return f"for j in range({n_times}):\n    {semicircle_code}\n    {semicircle_code}"

    def concentric_polygon(self, n_times: int, sides: int, length: int, left: bool = True) -> str:
        angle = 360 / sides
        direction = "left" if left else "right"
        return (
            f"for j in range({n_times}):\n    for i in range({sides}):\n        forward({length} * j)\n        {direction}({angle})"
        )


# Combination of shapes and positions
    def shape_sequence(self, shapes: str, left: bool = True) -> str:
        return "\n".join(shapes)

    def sub_program(self, shape_sequence: str, locals: bool = True) -> str:
        return f"embed(\"\"\"{shape_sequence}\"\"\", locals())"

    # Snowflake
    def snowflake(self, sides: int, sub_program: str, left: bool = True) -> str:
        direction = "left" if left else "right"
        return f"for j in range({sides}):\n    {sub_program}\n    forward(0)\n    {direction}({360 / sides})"

In [18]:
# Concentric circle
print(df_all['Program'].loc[209])
print("\n")
generator=generateLOGOPseudoCode()
print(generator.concentric_semicircle(4, semicircle=False, left=True))

for j in range(4):
    for i in range(HALF_INF):
        forward(EPS_DIST*j)
        left(EPS_ANGLE)
    for i in range(HALF_INF):
        forward(EPS_DIST*j)
        left(EPS_ANGLE)


for j in range(4):
    for i in range(180):
        forward(0.03490481287456702 * j)
        left(1.0)
    for i in range(180):
        forward(0.03490481287456702 * j)
        left(1.0)


In [ ]:
# seemeslike there is a rule the arms of a snowflake if they contain a space and or a line then this is executed before the other basic geometric shapes such as polygons and circles.
    # probably because these start with an for-loop which would also affect lines or shapes since they are not separated by a closed parenthesis or embed 
#Are several polygons and circle if they are as arms or foolowing each other in the main program (aka shape_sequence) somehow embedded in embed() functions or how is this done

# Should I impute sth for the semicircle variables: EPS_ANGLE, EPS_DIST, HALF_INF? 
# in the interpreter I used this
#    "EPS_ANGLE": 1.0,    # incrementing by 1° at each step   
#    "EPS_DIST": 0.03490481287456702, 
#    "HALF_INF": 180    # half-circle has 180°

# I might need to generate a parsing function so that i can run the generated programs in turtle seeing if they execute (Remember: the python turtle library doesn't run on the server)

#### Random generation of LOGO Programs

Randomly generate new LOGO Programs in pseudo code. (Ist sozusagen ein Stichprobe ohne zurücklegen von der Grundgesamtheit aller Möglichen LOGO Programme gegeben der Einschränkungen meiner LOGO Syntax)

In [24]:
import random
class LOGOProgramSampler:
    def __init__(self, generator_class):
        self.generator = generator_class
        self.generated = set()  # Track generated programs to avoid duplicates

    def _generate_random_program(self):
        """
        Generate a random program based on the grammar dynamically.
        """
        description = ""
        rule = random.choice(["Position", "Shape-Sequence", "SpecialShape", "Snowflake"])
        
        if rule == "Position":
            program, desc = self._generate_random_position()
        elif rule == "Shape-Sequence":
            program, desc = self._generate_random_shape_sequence()
        elif rule == "SpecialShape":
            program, desc = self._generate_random_special_shape()
        elif rule == "Snowflake":
            program, desc = self._generate_random_snowflake()
        
        description += desc
        return program, description

    def _generate_random_position(self):
        """
        Generate a random position program.
        """
        description = ""
        position_type = random.choice(["InARow", "Concentric"])
        
        if position_type == "InARow":
            sub_program, sub_desc = self._generate_random_shape()
            n_times = random.randint(1, 9)
            sub_desc = sub_desc.removeprefix("a ")
            description += f"{n_times} {sub_desc} in a row"
            return self.generator.in_a_row(n_times, sub_program), description
        
        elif position_type == "Concentric":
            shape_type = random.choice(["semicircle", "polygon"])
            n_times = random.randint(2, 9)
            if shape_type == "semicircle":
                description += f"{n_times} concentric circles"
                return self.generator.concentric_semicircle(n_times, semicircle=False), description
            else:
                sides = random.randint(3, 9)
                length = random.choice([2, 4, 20])
                size_desc = "small" if length == 2 else "medium" if length == 4 else "big"
                shape_desc = "triangle" if sides == 3 else "square" if sides == 4 else f"{sides}-gon"
                description += f"{n_times} concentric {size_desc} {shape_desc}"
                return self.generator.concentric_polygon(n_times, sides, length), description

    def _generate_random_shape_sequence(self, snowflake_arm: bool = False):
        """
        Generate a random sequence of shapes.
        """
        if snowflake_arm:
            n_shapes = random.randint(2, 5) # snowflake arms can also start with a space
        else:
            n_shapes = random.randint(3, 5) # normal shape-sequences shouldn't start with a space and to deserve being described to be connected or separated they have to consist of at least 3 shapes
        shapes = []
        descriptions = []
        connected_separated = random.choice([True, False])
        
        for n in range(n_shapes):
            if connected_separated:
                shape, desc = self._generate_random_shape()
            else:
                if not snowflake_arm and n == 0:                # if it is not the arm of a snowflake,
                    shape, desc = self._generate_random_shape() # the first element is a shape
                elif n == n_shapes - 1:
                    shape, desc = self._generate_random_shape() # and the last element is always a shape
                else:
                    if any("space" in d for d in descriptions) or random.choice([True, False]):
                        shape, desc = self._generate_random_shape()
                    else:
                        length = random.choice([2, 4, 20])
                        angle = random.choice([0, 90, 180, 270])
                        shape = self.generator.generate_space(length, angle)
                        size_desc = "short" if length == 2 else "medium" if length == 4 else "big"
                        desc = f"a {size_desc} space"
            shapes.append(shape)
            descriptions.append(desc)
        
        description = f"{'connected' if connected_separated else 'separated'} sequence of shapes: " + ", ".join(descriptions)
        return self.generator.shape_sequence(shapes), description

    def _generate_random_shape(self):
        """
        Generate a random shape.
        """
        options = ["Line", "Polygon", "Semicircle"]
        weights = [1/10, 7/10, 2/10]
        shape_type = random.choices(population=options, weights=weights, k=1)[0]
        description = ""

        if shape_type == "Line":
            length = random.choice([2, 4, 20])
            angle = random.choice([0, 90, 180, 270])
            size_desc = "short" if length == 2 else "medium" if length == 4 else "big"
            description += f"a {size_desc} line"
            return self.generator.generate_line(length, angle), description
        
        elif shape_type == "Polygon":
            sides = random.randint(3, 9)
            length = random.choice([2, 4, 20])
            size_desc = "small" if length == 2 else "medium" if length == 4 else "big"
            shape_desc = "triangle" if sides == 3 else "square" if sides == 4 else f"{sides}-gon"
            description += f"a {size_desc} {shape_desc}"
            return self.generator.generate_polygon(sides, length), description
        
        elif shape_type == "Semicircle":
            semicircle = random.choice([True, False])
            size = random.choice([1, 2])
            size_desc = "small" if size == 1 else "medium"
            shape_desc = "semicircle" if semicircle else "circle"
            description += f"a {size_desc} {shape_desc}"
            return self.generator.generate_semicircle(size, semicircle=semicircle), description
    
    def _generate_random_special_shape(self):
        """
        Generate a random special shape.
        """
        special_shape_type = random.choice(["GreekSpiral", "Staircase", "Zigzag", "Star"])
        description = ""
        
        if special_shape_type == "GreekSpiral":
            size = random.randint(5, 9)
            description += f"a greek spiral with {size} turns"
            return self.generator.generate_greek_spiral(size), description
        elif special_shape_type == "Staircase":
            n_times = random.randint(2, 9)
            length = random.choice([2, 4, 20])
            size_desc = "small" if length == 2 else "medium" if length == 4 else "big"
            description += f"a staircase with {n_times} {size_desc} steps"
            return self.generator.generate_staircase(n_times, length), description
        elif special_shape_type == "Zigzag":
            n_times = random.randint(2, 9)
            length = random.choice([2, 4, 20])
            size_desc = "small" if length == 2 else "medium" if length == 4 else "big"
            description += f"a zigzag with {n_times} {size_desc} steps"
            return self.generator.generate_zigzag(n_times, length), description
        elif special_shape_type == "Star":
            sides = random.randint(5, 9)
            description += f"a star with {sides} points"
            return self.generator.generate_star(sides), description

    def _generate_random_snowflake(self):
        """
        Generate a random snowflake program.
        """
        sides = random.randint(3, 8)
        n_arms = random.randint(1, 2)
        description = f"a {sides} sided snowflake with "

        if n_arms == 1:
            arm , arm_description = self._generate_random_shape()   # contains a single shape
            description += f"an arm of {arm_description.lower()}"

        else:
            arm, arm_description = self._generate_random_shape_sequence(snowflake_arm=True) # contains multiple shapes
            description += f"arms of {arm_description.lower()}"

        sub_program = self.generator.sub_program(arm)
        return self.generator.snowflake(sides, sub_program), description

    def sample(self, n=1):
        """
        Dynamically generate `n` unique random programs.
        """
        samples = []
        while len(samples) < n:
            program, description = self._generate_random_program()
            if program not in self.generated:
                self.generated.add(program)
                samples.append({"Program": program, "Description": description})
        return samples

    def reset(self):
        """
        Reset the sampler to allow generating duplicates.
        """
        self.generated = set()


In [25]:
sampler = LOGOProgramSampler(generateLOGOPseudoCode())
sampled_data = sampler.sample(5)

# Print the synthetic data
for item in sampled_data:
    print(f"Program: {item['Program']}")
    print(f"Description: {item['Description']}")
    print()

Program: for j in range(8):
    embed("""for i in range(3):
    forward(20)
    left(120.0)""", locals())
    forward(0)
    left(45.0)
Description: a 8 sided snowflake with an arm of a big triangle

Program: for j in range(2):
    for i in range(7):
        forward(2 * j)
        left(51.42857142857143)
Description: 2 concentric small 7-gon

Program: for i in range(7):
    forward(10)
    left(154.28571428571428)
Description: a star with 7 points

Program: for j in range(6):
    embed("""for i in range(180):
    forward(0.06980962574913405)
    left(1.0)
for i in range(180):
    forward(0.06980962574913405)
    left(1.0)
for i in range(8):
    forward(20)
    left(45.0)
for i in range(180):
    forward(0.06980962574913405)
    left(1.0)
for i in range(180):
    forward(0.06980962574913405)
    left(1.0)
for i in range(5):
    forward(4)
    left(72.0)""", locals())
    forward(0)
    left(60.0)
Description: a 6 sided snowflake with arms of connected sequence of shapes: a medium circle

#### Interpreter for pseudo code generating LOGO graphics

The purpose is to define the LOGO functions and an envirnoment in which they can be executed producing graphic output.
This is neccessary to validate that the programs produce valid graphics and to have an output which can be ASCII-transformed and fed into the LLM later on.



In [6]:
import matplotlib.pyplot as plt
import numpy as np

# Implementation of ReGAL LOGO primitives using Matplotlib
class ReGALLOGOPrimitives:
    '''Defining all the LOGO ReGAL primitives'''

    def __init__(self):
        self.x, self.y = 0, 0  # Current position
        self.angle = 0         # Current angle in degrees
        self.is_drawing = True
        self.path = []         # List of drawn lines
        self.pen_up_path = []  # List of pen-up moves

    def _add_to_path(self, x1, y1, x2, y2):
        if self.is_drawing:
            self.path.append(((x1, y1), (x2, y2)))
        else:
            self.pen_up_path.append(((x1, y1), (x2, y2)))

    def forward(self, distance):
        x2 = self.x + distance * np.cos(np.radians(self.angle))
        y2 = self.y + distance * np.sin(np.radians(self.angle))
        self._add_to_path(self.x, self.y, x2, y2)
        self.x, self.y = x2, y2

    def left(self, angle):
        self.angle = (self.angle + angle) % 360

    def right(self, angle):
        self.angle = (self.angle - angle) % 360

    def penup(self):
        self.is_drawing = False

    def pendown(self):
        self.is_drawing = True

    def teleport(self, x, y):
        self.penup()
        self.x, self.y = x, y
        self.pendown()

    def heading(self, angle):
        self.angle = angle % 360

    def isdown(self):
        return self.is_drawing

# Interpreter class
class PseudoProgramInterpreter:
    def __init__(self):
        self.state = ReGALLOGOPrimitives()
        self.circle_vars = {
            "EPS_ANGLE": 1.0,    # incrementing by 1° at each step   
            "EPS_DIST": 0.03490481287456702, 
            "HALF_INF": 180    # half-circle has 180°        
            }

    def execute(self, program, local_vars=None):
        """
        Execute a program sequence.
        program: str, the program text
        local_vars: dict, the local variables for `embed` functionality
        """
        local_scope = {"forward": self.state.forward,
                       "left": self.state.left,
                       "right": self.state.right,
                       "penup": self.state.penup,
                       "pendown": self.state.pendown,
                       "teleport": self.state.teleport,
                       "heading": self.state.heading,
                       "isdown": self.state.isdown,
                       "embed": self.embed}
        
        # Update the local scope with circle variables
        local_scope.update(self.circle_vars)  # Add circle variables
        if local_vars:
            local_scope.update(local_vars)
        
        #print(f"Executing program: {program}") # Debugging
        exec(program, {}, local_scope)

    def embed(self, subprogram, local_vars):
        """
        Executes an embedded subprogram with access to the given locals.
        """
        self.execute(subprogram, local_vars)

    def save_graphics(self, filename="output.png"):
        """
        Saves the generated graphics as an image file.
        """
        fig, ax = plt.subplots()
        ax.set_aspect('equal', adjustable='datalim') # this I need to adjust so that small graphics also come accross as smaller then large graphics 

        # Draw paths
        for (x1, y1), (x2, y2) in self.state.path:
            ax.plot([x1, x2], [y1, y2], 'k-')  # Pen-down lines in black

        # Draw pen-up paths for visualization (optional)
        for (x1, y1), (x2, y2) in self.state.pen_up_path:
            ax.plot([x1, x2], [y1, y2], 'r--', alpha=0.0)  # Pen-up lines in dashed red (if alpha > 0.0)

        plt.axis('off')  # Hide axes
        plt.savefig(filename, bbox_inches='tight')
        plt.close(fig)
    
    def reset_state(self):
        """
        Resets the graphics state for a new drawing.
        """
        self.state = ReGALLOGOPrimitives()

In [7]:
# TEST 
import os
# Create directory for saving graphics
output_dir = "logo_graphic/11testshapes"
os.makedirs(output_dir, exist_ok=True)

# create a test dataset form the all_data
test_indices = [98, 44, 100, 99, 200, 212, 214, 201, 53, 54, 282]
df_test_subset = df_all.loc[test_indices].reset_index(drop=True)
display(df_test_subset)

# Load dataset and generate graphics
interpreter = PseudoProgramInterpreter()

for index, row in df_test_subset.iterrows():
    human_message = row['Description']
    gpt_message = row['Program']
    filename = f"{index}_{human_message.replace(' ', '_')}.png"
    filepath = os.path.join(output_dir, filename)
    
    interpreter.reset_state()

    # Execute program and save the graphic
    interpreter.execute(gpt_message)
    interpreter.save_graphics(filepath)

,Description,Program
0,a medium 8 gon,for i in range(8):\n forward(4)\n left(4...
1,a small 7 gon separated by a big space from a ...,for i in range(7):\n forward(2)\n left(5...
2,8 concentric circle s,for j in range(9):\n for i in range(HALF_IN...
3,5 short line s in a row,"for j in range(5):\n embed(""""""forward(2)\nl..."
4,a greek spiral with 7 turns,for i in range(8):\n forward(1*i)\n left...
5,a 3 stepped staircase,for i in range(3):\n forward(2)\n left(9...
6,a 3 stepped zigzag,forward(0)\nleft(45.0)\n\nfor i in range(3):\n...
7,a 7 pointed star,for i in range(7):\n forward(16)\n left(...
8,6 sided snowflake with a small square as arms,"for j in range(6):\n embed(""""""for i in rang..."
9,5 sided snowflake with a medium line and a sma...,"for j in range(5):\n embed(""""""forward(8)\nl..."


In [9]:
# ReGAL DATA GRAPHICS
import os
# Create directory for saving graphics
output_dir = "logo_graphic/train200_test"
os.makedirs(output_dir, exist_ok=True)
#df_all_test=df_all.head(10)

# Load dataset and generate graphics
interpreter = PseudoProgramInterpreter()

for index, row in df_all.iterrows():
    human_message = row['Description']
    gpt_message = row['Program']
    filename = f"{index}_{human_message.replace(' ', '_')}.png"
    filepath = os.path.join(output_dir, filename)
    
    interpreter.reset_state()

    # Execute program and save the graphic
    interpreter.execute(gpt_message)
    interpreter.save_graphics(filepath)

Executing program: for i in range(5):
    embed("""for j in range(4):
    forward(2*i)
    left(90.0)""", locals())
Executing program: for j in range(4):
    forward(2*i)
    left(90.0)
Executing program: for j in range(4):
    forward(2*i)
    left(90.0)
Executing program: for j in range(4):
    forward(2*i)
    left(90.0)
Executing program: for j in range(4):
    forward(2*i)
    left(90.0)
Executing program: for j in range(4):
    forward(2*i)
    left(90.0)
Executing program: for j in range(6):
    embed("""forward(8)
left(0.0)
for i in range(5):
    forward(4)
    left(72.0)""", locals())
    forward(0)
    left(60.0)

Executing program: forward(8)
left(0.0)
for i in range(5):
    forward(4)
    left(72.0)
Executing program: forward(8)
left(0.0)
for i in range(5):
    forward(4)
    left(72.0)
Executing program: forward(8)
left(0.0)
for i in range(5):
    forward(4)
    left(72.0)
Executing program: forward(8)
left(0.0)
for i in range(5):
    forward(4)
    left(72.0)
Executing pr

In [15]:
# SYNTEHTIC DATA GRAPHICS
# Executing the programs generated by the pseudo code generator
import os
generator=generateLOGOPseudoCode()
star=generator.generate_star(6)
output_dir = "logo_graphic/synthetic"
os.makedirs(output_dir, exist_ok=True)
name = f"star"

interpreter = PseudoProgramInterpreter()
interpreter.reset_state()

# Execute program and save the graphic
#triangle4_in_row = triangle4_in_row
interpreter.execute(star)
interpreter.save_graphics(os.path.join(output_dir, name + ".png"))